In [9]:
#generate for bar chart
#from gensim.models.keyedvectors import KeyedVectors
#from gensim.models import word2vec
import json
import numpy as np
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math
import csv
import codecs
l_quality= open("lecture_quality.txt", "r", encoding="utf-8").readlines()
load= open("load.txt", "r", encoding="utf-8").readlines()
grade= open("grade.txt", "r", encoding="utf-8").readlines()
int_friendly= open("international_friendly.txt", "r", encoding="utf-8").readlines()
CATEGORY_COUNT = 3

In [10]:
pos_vectors = KeyedVectors.load_word2vec_format('../pos_naver_news.vec', binary=False)

NameError: name 'KeyedVectors' is not defined

In [11]:
scores = []
subjects = [l_quality, load, grade]
def add_to_dict(subject):
    subject_scores={}
    for i in range(len(subject)):
        p = subject[i].split("\t")
        subject_scores[p[0]] = float(p[1][:-1])
    scores.append(subject_scores)

for subject in subjects:
    add_to_dict(subject)

In [12]:
subject_mp = {"강의":0,"널널":1,"성적":2}
def calculate_score(s, subject_str):
    result = 0
    for k,v in scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

In [13]:
REVIEWS = "cs_reviews"

f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

scores_list = []
subjects_str = ["강의", "널널", "성적"]
for subject_str in subjects_str:
    scores_list_subject=[]
    for i in range(len(f[:])):
        line = f[i]
        course = json.loads(line)
        scores_list_subject.append(calculate_score(course["review_text"], subject_str))
    scores_list.append(scores_list_subject)


In [32]:
f_count_word = {}
count_word = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
#num_word={}

def calc_weight(s):
    return 3 - (mp[s])

for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["성적"] == "?":
        continue
    words = course["review_text"].split()
    for w in words:
        if w not in count_word:
            count_word[w] = 0
            #num_word[w]=0
        #num_word[w]+=1
        count_word[w] += calc_weight(course["성적"])
# for w in count_word:
#     count_word[w]=float(count_word[w])/float(num_word[w])


In [33]:
sorted_by_value = sorted(count_word.items(), key=lambda kv: kv[1])[::-1]
print(sorted_by_value)

[('관리', 4), ('학교에서', 4), ('불만족스러웠습니다.', 4), ('지질게요.', 4), ('장을', 4), ('결코', 3), ('과목포기함', 2), ('중간고사쳤다가', 2), ('패기넘치게', 2), ('띠바', 2), ('모르겠는', 2), ('변화가', 2), ('혁신적인', 2), ('보낸', 2), ('받아야', 2), ('코멘트들을', 2), ('받았는데,', 2), ('박사분께', 2), ('문서나', 2), ('발전이', 2), ('capstone', 2), ('것들로', 2), ('현실에', 2), ('해외에선', 2), ('대표적인', 2), ('따라가지', 2), ('현실을', 2), ('하더군요.', 2), ('방법이라고', 2), ('쓰였던', 2), ('전에나', 2), ('20-30년', 2), ('경험자에게', 2), ('실무', 2), ('인턴십', 2), ('좌절하게', 2), ('실무에서도', 2), ('시작을', 2), ('작성하느라', 2), ('requirements/design', 2), ('8주', 2), ('하라고', 2), ('16주에', 2), ('문서의', 2), ('않았을', 2), ('아니었다면', 2), ('수강했던', 2), ('재학', 2), ('상태가', 2), ('학점의', 2), ('추천.', 2), ('피하시는', 2), ('싫어하시면', 2), ('많이하는', 2), ('배신하는', 2), ('많이해야하는데', 2), ('노력은', 2), ('것입니다)로드는', 2), ('글', 2), ('(밑에', 2), ('자체가....', 2), ('준것', 2), ('B-정도', 2), ('median', 2), ('준지', 2), ('몇명', 2), ('ㅠ)', 2), ('들었다가', 2), ('(80명이', 2), ('뿐이었다고..', 2), ('2명', 2), ('수강생중', 2), ('A+가', 2), ('따르면', 2), ('풍문에', 2), ('많았음에도,', 2), ('

In [16]:
# sorted_by_value

In [17]:
def calc_score_avg(column_name,comment_id):
    i = comment_id
    course = json.loads(f[i])
    #print(mp[course["강의"]],(scores_list[i]))
    return scores_list[subject_mp[column_name]][i]
    

for i in range(len(f)):
    course = json.loads(f[i])
    print(course["course_code"])
    
    for x in subjects_str:
        print(x,calc_score_avg(x,i))
    break
    

CS322
강의 0.0996679946249559
널널 -0.761594155955765
성적 -0.8336546070121552


In [18]:
def append_noun(s):
    return "('%s','Noun')" % (s)

naver_wv_categories = ["강의", "부담", "성적"]
wv_update = {}
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        if append_noun(k) in pos_vectors.vocab:
#             print(k,naver_wv_categories[i])
#             print(pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i])))
            wv_update[k] = pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i]))
        

NameError: name 'pos_vectors' is not defined

In [ ]:
review_update = {}
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)

    for i in range(CATEGORY_COUNT):
        if course[subjects_str[i]] == "?":
            continue
        for k,v in scores[i].items():
            if k in course["review_text"].split():
                if k not in review_update:
                    review_update[k] = 0
                review_update[k] += calc_weight(course[subjects_str[i]])*100.0/len(course["review_text"])
        

In [ ]:
aggregated_scores = [{} for i in range(CATEGORY_COUNT)]
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        aggregated_scores[i][k] = v
        if k in review_update:
            aggregated_scores[i][k] += review_update[k]*0.1
        
        if k in wv_update:
            aggregated_scores[i][k] += wv_update[k]
            
            

In [ ]:
def calc_aggregated_score(s, subject_str):
    result = 0
    for k,v in aggregated_scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

def calc_aggregated_score_sum(column_name,i):
    course = json.loads(f[i])
    return calc_aggregated_score(course["review_text"], column_name)

# print(subjects_str)

In [ ]:
for i in range(len(f[:1])):
    course = json.loads(f[i])
    print(course)

In [ ]:
pos = {}
neg = {}
PNS = {}
course_list = []
print(subjects_str)

for i in range(len(f[:])):
    course = json.loads(f[i])
#     print(course["course_code"]+"\t",end="")
    if course["course_code"] not in pos:
        pos[course["course_code"]] = [[0,0],[0,0],[0,0]]
        neg[course["course_code"]] = [[0,0],[0,0],[0,0]]
        PNS[course["course_code"]] = [[0,0],[0,0],[0,0]]#[0,0,0]
        course_list.append(course["course_code"])
    for x in subjects_str:
#             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
#         print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
        sc = calc_aggregated_score_sum(x,i)
        if sc>0:
            pos[course["course_code"]][subject_mp[x]][0] += sc*10
            pos[course["course_code"]][subject_mp[x]][1] += 1
        else:
            neg[course["course_code"]][subject_mp[x]][0] -= sc*10
            neg[course["course_code"]][subject_mp[x]][1] += 1
    print("")
    for x in subjects_str:
#         pos_vir = pos[course["course_code"]][subject_mp[x]][1]
#         neg_vir = neg[course["course_code"]][subject_mp[x]][1]
#         if pos_vir == 0 and neg_vir == 0:
#             pos_vir = 1
        pos_sum = pos[course["course_code"]][subject_mp[x]][0]
#         pos_cnt = pos[course["course_code"]][subject_mp[x]][1]
        neg_sum = neg[course["course_code"]][subject_mp[x]][0]
#         neg_cnt = neg[course["course_code"]][subject_mp[x]][1]
        PNS[course["course_code"]][subject_mp[x]][0] = pos_sum-neg_sum

In [ ]:
print(neg["CS322"])

In [19]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos.items():
        print(k)
        print(*pos[k])
        print(*neg[k])
        print(*PNS[k])
    
    

NameError: name 'pos' is not defined

In [ ]:
for k,v in pos.items():
    with open("../data/"+k+".csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(["Course","Pos","Neg","PNS"])
        for subject in subjects_str :
            if subject == "강의":
                line = ["Lecture",str(pos[k][0][0]),str(neg[k][0][0]),str(PNS[k][0][0])]
            elif subject == "널널":
                line = ["Load",str(pos[k][1][0]),str(neg[k][1][0]),str(PNS[k][1][0])]
            else:
                line = ["Grade",str(pos[k][2][0]),str(neg[k][2][0]),str(PNS[k][2][0])]
            writer.writerow(line)

In [20]:
for i in range(len(f[:])):
    course = json.loads(f[i])
    str1 = course["course_code"] + course["year"] + course["semester"]
    print(str1)
    break
    

CS3222012가을


In [21]:
pos1 = {}
neg1 = {}
PNS1 = {}

for i in range(len(f[:])):
    course = json.loads(f[i])
    key1 = course["course_code"] +" "+ course["year"] +" "+ course["semester"]
    if course["semester"]!="봄" and course["semester"]!="가을":
        pass
    else:
        print(course["course_code"]+"\t",end="")
        if key1 not in pos1:
            pos1[key1] = [[0,0],[0,0],[0,0]]
            neg1[key1] = [[0,0],[0,0],[0,0]]
            PNS1[key1] = [[0,0],[0,0],[0,0]]#[0,0,0]
        for x in subjects_str:
    #             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
            print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
            sc = calc_aggregated_score_sum(x,i)
            if sc>0:
                pos1[key1][subject_mp[x]][0] += sc*10
                pos1[key1][subject_mp[x]][1] += 1
            else:
                neg1[key1][subject_mp[x]][0] -= sc*10
                neg1[key1][subject_mp[x]][1] += 1
        print("")
        for x in subjects_str:
            pos_vir1 = pos1[key1][subject_mp[x]][1]
            neg_vir1 = neg1[key1][subject_mp[x]][1]
            if pos_vir1 == 0:
                pos_vir1 = 1
            if neg_vir1 == 0:
                neg_vir1 = 1
            pos_sum1 = pos1[key1][subject_mp[x]][0]
            pos_cnt1 = pos1[key1][subject_mp[x]][1]
            neg_sum1 = neg1[key1][subject_mp[x]][0]
            neg_cnt1 = neg1[key1][subject_mp[x]][1]
            PNS1[key1][subject_mp[x]][0] = pos_sum1-neg_sum1
            
            
            

CS322	

NameError: name 'calc_aggregated_score_sum' is not defined

In [ ]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos1.items():
        print(k)
        print(v)

In [ ]:
#season : 11:13 course:0:5, Year:6:10
# course_dict ={} #key:name, value: dict->key:year+semester val:PNS
# for k,v in PNS.items():
#     name = k[0:5]
#     if name not in course_dict:
#         course_dict[name][k[6:]]=v
# print(course_dict)
# for k1,v1 in course_dict.items():
#     #with codecs.open("../data/line/"+k[0:5]+" line.csv","wb","utf-8-sig") as file:
#     #writer = csv.writer(file, delimiter=',')
#     L = [["Categories","Lecture","Load","Grade"]]
#     for k2,v2 in v1.items():
#         L.append([k2,v2[0][0],v2[1][0],v2[2][0]])
# print(L)

In [ ]:
for course_tmp in course_list:
    L =[["Categories","Lecture","Load","Grade"]]
    for i in range(2000,2019):
        for k1,v1 in PNS1.items():
            if k1[0:5] ==course_tmp and k1[6:10] == str(i):
                if k1[11:] == "봄":
                    L.append([k1[6:10]+" Spring",v1[0][0],v1[1][0],v1[2][0]])
                if k1[11:] == "가을":
                    L.append([k1[6:10]+" Fall",v1[0][0],v1[1][0],v1[2][0]])    
    if(len(L)>2):
        for idx in range(1,len(L)-1):
            if int(L[idx][0][0:4])==int(L[idx+1][0][0:4]) and L[idx][0][5:]=="Fall" and L[idx+1][0][5:]=="Spring":
                tmp = L[idx]
                L[idx] = L[idx+1]
                L[idx+1] = tmp
    with open("../data/line/"+course_tmp+" line.csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for i in range(len(L)):
            writer.writerow(L[i])
        
        